# Logistic regression on Average Weight based on game mechanisms

I import all python packages

In [6]:
from sql_functions import get_dataframe
import pandas as pd
import numpy as np
# Import packages 
import numpy as np
import pandas as pd

from timeit import default_timer as timer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

I import the stats and mechanisms

In [7]:
schema = "bgg_data"
main_stats = "unfiltered_main_stats_cleaned"

df_main_stats = get_dataframe(f"SELECT * FROM {schema}.{main_stats}")

df_mechanics = get_dataframe(f'select * from {schema}.mechanics')

df_designer = get_dataframe(f"select * from {schema}.designer")

In [8]:
df_designer.groupby('designer_id').count().sort_values('id',ascending=False)

,id
designer_id,
3,19431
810,924
2,671
1533,571
8693,331
...,...
5969,1
5970,1
66171,1


In [9]:
df_designer

,id,designer_id
0,98401,42353
1,98402,28161
2,98406,35005
3,98416,26845
4,98417,3
...,...,...
147944,298190,20991
147945,298191,123709
147946,298191,123707
147947,298193,119884


In [10]:
mask = df_designer['id'] == 3

In [11]:
mask = df_main_stats['numweights'] > 5

df_main_stats = df_main_stats.loc[mask,:]

In [12]:
df_main_stats.drop(['yearpublished','min_players',"max_players","playtime","max_playtime","average","user_rated",'num_owned',"trading",'wanting','wishing',"numcomments","numweights"],axis=1,inplace=True)

In [13]:
df_main_stats

,id,min_playtime,min_age,averageweight
17,98443,90.0,10,2.6667
25,98472,60.0,10,2.2143
37,98527,60.0,13,3.0988
38,98529,15.0,6,1.0909
68,51314,20.0,13,2.8333
...,...,...,...,...
137093,298069,30.0,10,2.1685
137100,298102,45.0,11,2.1250
137116,298130,120.0,12,3.1429
137130,298154,120.0,14,4.3750


In [14]:
df_merged_temp = pd.merge(df_main_stats,df_mechanics,on='id')
df_merged = pd.merge(df_merged_temp,df_designer,on='id')

In [15]:
df_merged

,id,min_playtime,min_age,averageweight,mechanic_id,designer_id
0,98443,90.0,10,2.6667,2072,1642
1,98443,90.0,10,2.6667,2081,1642
2,98443,90.0,10,2.6667,2015,1642
3,98443,90.0,10,2.6667,2082,1642
4,98443,90.0,10,2.6667,2935,1642
...,...,...,...,...,...,...
88457,298154,120.0,14,4.3750,2012,49282
88458,298154,120.0,14,4.3750,2005,49282
88459,298154,120.0,14,4.3750,2008,49282
88460,298166,5.0,8,1.1250,2072,113378


## Create a dummies matrix

In [16]:
dummies_matrix = pd.get_dummies(df_merged['mechanic_id'])


dummies_matrix_2 = pd.get_dummies(df_merged['designer_id'])

In [17]:
dummies_merged = pd.merge(dummies_matrix,dummies_matrix_2,left_index=True,right_index=True)

In [18]:
dummies_merged

,2001,2002_x,2003,2004,2005,2007,2008_x,2009,2010,2011,...,144166,144167,144474,144543,145347,145478,145479,145532,145851,145972
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88458,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88459,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:

merged_with_dummies = df_merged.join(dummies_merged)

In [20]:
merged_with_dummies

,id,min_playtime,min_age,averageweight,mechanic_id,designer_id,2001,2002_x,2003,2004,...,144166,144167,144474,144543,145347,145478,145479,145532,145851,145972
0,98443,90.0,10,2.6667,2072,1642,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,98443,90.0,10,2.6667,2081,1642,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,98443,90.0,10,2.6667,2015,1642,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,98443,90.0,10,2.6667,2082,1642,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,98443,90.0,10,2.6667,2935,1642,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88457,298154,120.0,14,4.3750,2012,49282,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88458,298154,120.0,14,4.3750,2005,49282,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88459,298154,120.0,14,4.3750,2008,49282,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88460,298166,5.0,8,1.1250,2072,113378,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
merged_with_dummies.drop(3,axis=1,inplace=True)

In [23]:
# Q1–1.5 IQR
# Q3+1.5 IQR

min_pt_iqr = merged_with_dummies['min_playtime'].quantile(0.75) - merged_with_dummies['min_playtime'].quantile(0.25)

min_pt_iqr

60.0

## Remove outliers with interquartile range

In [24]:
mask =  (merged_with_dummies['min_playtime'] < merged_with_dummies['min_playtime'].quantile(0.75) + 1.5 * min_pt_iqr) & (merged_with_dummies['min_playtime'] > 5)

merged_with_dummies_clean = merged_with_dummies.loc[mask,:]

In [25]:
merged_with_dummies_clean = merged_with_dummies_clean.groupby('id').max()

In [26]:
merged_with_dummies_clean

,min_playtime,min_age,averageweight,mechanic_id,designer_id,2001,2002_x,2003,2004,2005,...,144166,144167,144474,144543,145347,145478,145479,145532,145851,145972
id,,,,,,,,,,,,,,,,,,,,,
2,30.0,12,1.9630,2009,8384,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30.0,10,2.4836,2846,2,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,60.0,12,2.6667,2080,8008,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,90.0,12,2.5009,2940,4,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,20.0,8,1.7957,2060,1335,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359438,75.0,12,3.6667,2910,11767,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
359871,60.0,14,2.7000,2856,35585,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
360226,60.0,13,2.5556,2933,144474,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [27]:
merged_with_dummies_clean.drop(['mechanic_id','designer_id'],axis=1,inplace=True)

In [28]:
merged_with_dummies_clean_alt = merged_with_dummies_clean.copy()

## Create a category

In [29]:
merged_with_dummies_clean['averageweight_bins'] = pd.cut(merged_with_dummies_clean['averageweight'],bins=[1,2,3,4,5],labels=['1,5','2.5','3.5','4.5'])

In [30]:
merged_with_dummies_clean.drop('averageweight',axis=1,inplace=True)

In [31]:
merged_with_dummies_clean

,min_playtime,min_age,2001,2002_x,2003,2004,2005,2007,2008_x,2009,...,144167,144474,144543,145347,145478,145479,145532,145851,145972,averageweight_bins
id,,,,,,,,,,,,,,,,,,,,,
2,30.0,12,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"1,5"
3,30.0,10,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
4,60.0,12,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
5,90.0,12,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
7,20.0,8,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"1,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359438,75.0,12,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
359871,60.0,14,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2.5
360226,60.0,13,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.5


In [32]:
merged_with_dummies_clean.isnull().sum().sort_values(ascending=False)

averageweight_bins    430
66727                   0
67487                   0
67476                   0
67469                   0
                     ... 
6942                    0
6941                    0
6940                    0
6925                    0
28049                   0
Length: 7798, dtype: int64

In [33]:
merged_with_dummies_clean.dropna(axis=0,subset='averageweight_bins',inplace=True)

In [34]:
merged_with_dummies_clean

,min_playtime,min_age,2001,2002_x,2003,2004,2005,2007,2008_x,2009,...,144167,144474,144543,145347,145478,145479,145532,145851,145972,averageweight_bins
id,,,,,,,,,,,,,,,,,,,,,
2,30.0,12,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"1,5"
3,30.0,10,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
4,60.0,12,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
5,90.0,12,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
7,20.0,8,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"1,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359438,75.0,12,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
359871,60.0,14,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2.5
360226,60.0,13,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.5


In [35]:
merged_with_dummies_clean.describe()

,min_playtime,min_age,2001,2002_x,2003,2004,2005,2007,2008_x,2009,...,144166,144167,144474,144543,145347,145478,145479,145532,145851,145972
count,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,...,14375.000000,14375.000000,14375.000000,14375.000000,14375.000000,14375.0,14375.0,14375.000000,14375.000000,14375.000000
mean,52.011478,10.185878,0.075826,0.104139,0.007165,0.155270,0.012591,0.038330,0.034365,0.016278,...,0.000070,0.000070,0.000070,0.000070,0.000070,0.0,0.0,0.000070,0.000070,0.000070
std,32.749836,3.323910,0.264729,0.305452,0.084347,0.362174,0.111506,0.191999,0.182172,0.126548,...,0.008341,0.008341,0.008341,0.008341,0.008341,0.0,0.0,0.008341,0.008341,0.008341
min,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
25%,30.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
50%,45.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
75%,60.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
max,160.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000


In [36]:
mask = merged_with_dummies_clean['min_age'] == 0

merged_with_dummies_clean.loc[mask,'min_age'] = np.nan

In [37]:
merged_with_dummies_clean['min_age'] = merged_with_dummies_clean['min_age'].fillna(merged_with_dummies_clean['min_age'].median())

In [38]:
df = merged_with_dummies_clean.copy()

In [39]:
df

,min_playtime,min_age,2001,2002_x,2003,2004,2005,2007,2008_x,2009,...,144167,144474,144543,145347,145478,145479,145532,145851,145972,averageweight_bins
id,,,,,,,,,,,,,,,,,,,,,
2,30.0,12.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"1,5"
3,30.0,10.0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
4,60.0,12.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
5,90.0,12.0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
7,20.0,8.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"1,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359438,75.0,12.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
359871,60.0,14.0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2.5
360226,60.0,13.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.5


### I want to add a weight to the min_age multiplying min age x weight

In [40]:
df['min_playtime'] = (df['min_playtime'].max() - df['min_playtime'])/(df['min_playtime'].max()-df['min_playtime'].min())
df['min_age'] = (df['min_age'].max() - df['min_age'])/(df['min_age'].max()-df['min_age'].min())

In [41]:
df

,min_playtime,min_age,2001,2002_x,2003,2004,2005,2007,2008_x,2009,...,144167,144474,144543,145347,145478,145479,145532,145851,145972,averageweight_bins
id,,,,,,,,,,,,,,,,,,,,,
2,0.844156,0.500000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"1,5"
3,0.844156,0.611111,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
4,0.649351,0.500000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
5,0.454545,0.500000,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
7,0.909091,0.722222,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"1,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359438,0.551948,0.500000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
359871,0.649351,0.388889,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2.5
360226,0.649351,0.444444,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.5


In [42]:
df.groupby('averageweight_bins').count()

,min_playtime,min_age,2001,2002_x,2003,2004,2005,2007,2008_x,2009,...,144166,144167,144474,144543,145347,145478,145479,145532,145851,145972
averageweight_bins,,,,,,,,,,,,,,,,,,,,,
"1,5",6406,6406,6406,6406,6406,6406,6406,6406,6406,6406,...,6406,6406,6406,6406,6406,6406,6406,6406,6406,6406
2.5,6002,6002,6002,6002,6002,6002,6002,6002,6002,6002,...,6002,6002,6002,6002,6002,6002,6002,6002,6002,6002
3.5,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,...,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749
4.5,218,218,218,218,218,218,218,218,218,218,...,218,218,218,218,218,218,218,218,218,218


# Define predictors and targets

In [43]:
y = df.averageweight_bins
X = df.drop('averageweight_bins', axis=1)

In [44]:
y.head()

id
2    1,5
3    2.5
4    2.5
5    2.5
7    1,5
Name: averageweight_bins, dtype: category
Categories (4, object): ['1,5' < '2.5' < '3.5' < '4.5']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

# Logistic regression

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
classifier = LogisticRegression(fit_intercept= True, penalty='l1', solver= 'saga') # instantiate a sklearn logistic regression classs
classifier.fit(X_train, y_train) # fit the classifier/model on our train data 
y_prediction = classifier.predict(X_test) # use the fit model to predict on our test data 

/opt/homebrew/Caskroom/miniconda/base/envs/neesql/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/neesql/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/neesql/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [48]:
cm = confusion_matrix(y_test, y_prediction) # assign a confusion matrix that compares test data and predictions 
cm

array([[1058,  217,    6,    0],
       [ 243,  892,   64,    1],
       [  21,  196,  132,    1],
       [   0,   19,   12,   13]])

In [49]:
print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

         1,5       0.80      0.83      0.81      1281
         2.5       0.67      0.74      0.71      1200
         3.5       0.62      0.38      0.47       350
         4.5       0.87      0.30      0.44        44

    accuracy                           0.73      2875
   macro avg       0.74      0.56      0.61      2875
weighted avg       0.73      0.73      0.72      2875



In [50]:
classifier.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l1',
 'random_state': None,
 'solver': 'saga',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [51]:
param_grid = {"solver" : ["lbfgs", "liblinear", "newton-cg’", "sag", "saga"],
              "penalty" : ["l2", "l1", "elasticnet"],
              "fit_intercept" : [True, False]
             }

# Instantiate gridsearch and define the metric to optimize 
gs = GridSearchCV(classifier, param_grid, scoring='accuracy',
                  cv=5, verbose=1)

# Fit gridsearch object to data. Also lets see how long it takes.
start = timer()
gs.fit(X_train, y_train)
end = timer()
gs_time = end-start

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/opt/homebrew/Caskroom/miniconda/base/envs/neesql/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/neesql/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniconda/base/envs/neesql/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all 

In [ ]:
# Best score
print('Best score:', round(gs.best_score_, 3))


# Best parameters
print('Best parameters:', gs.best_params_)